In [224]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [225]:
df = pd.read_csv('insurance.csv')

In [226]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [227]:
df['smoker'] .replace(('yes', 'no'), (1, 0), inplace=True)
df['sex'] .replace(('male', 'female'), (1, 0), inplace=True)
df = pd.get_dummies(df, columns=['region'], drop_first=True)

In [228]:
df = df.rename(columns={'sex':'sex_male'})

In [229]:
charge_true = 25000
df['charges'] = df['charges'].apply(lambda x: 1 if x > charge_true else 0)

In [230]:
df.head()

,age,sex_male,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0


In [231]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
normalize_cols = mms.fit_transform(df[['bmi', 'age', 'children']])
print(normalize_cols)

[[0.3212268  0.02173913 0.        ]
 [0.47914985 0.         0.2       ]
 [0.45843422 0.2173913  0.6       ]
 ...
 [0.56201238 0.         0.        ]
 [0.26472962 0.06521739 0.        ]
 [0.35270379 0.93478261 0.        ]]


In [232]:
scaled_df = df
scaled_df[['bmi', 'age', 'children']] = normalize_cols
print(scaled_df.loc[4])

age                 0.304348
sex_male            1.000000
bmi                 0.347592
children            0.000000
smoker              0.000000
charges             0.000000
region_northwest    1.000000
region_southeast    0.000000
region_southwest    0.000000
Name: 4, dtype: float64


In [233]:
scaled_df.describe()

,age,sex_male,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,0.461022,0.505232,0.395572,0.218984,0.204783,0.150224,0.242900,0.272048,0.242900
std,0.305434,0.500160,0.164062,0.241099,0.403694,0.357425,0.428995,0.445181,0.428995
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.195652,0.000000,0.278080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.456522,1.000000,0.388485,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.717391,1.000000,0.504002,0.400000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [234]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
z_score =scaler.fit_transform(scaled_df)
z_score

array([[-1.43876426, -1.0105187 , -0.45332   , ..., -0.56641788,
        -0.61132367,  1.76548098],
       [-1.50996545,  0.98959079,  0.5096211 , ..., -0.56641788,
         1.63579466, -0.56641788],
       [-0.79795355,  0.98959079,  0.38330685, ..., -0.56641788,
         1.63579466, -0.56641788],
       ...,
       [-1.50996545, -1.0105187 ,  1.0148781 , ..., -0.56641788,
         1.63579466, -0.56641788],
       [-1.29636188, -1.0105187 , -0.79781341, ..., -0.56641788,
        -0.61132367,  1.76548098],
       [ 1.55168573, -1.0105187 , -0.26138796, ...,  1.76548098,
        -0.61132367, -0.56641788]])

In [235]:
z_scaled_df = pd.DataFrame(z_score, columns=df.columns)
z_scaled_df.loc[2]

age                -0.797954
sex_male            0.989591
bmi                 0.383307
children            1.580926
smoker             -0.507463
charges            -0.420453
region_northwest   -0.566418
region_southeast    1.635795
region_southwest   -0.566418
Name: 2, dtype: float64

In [236]:
X= scaled_df.drop('charges', axis=1)
y= scaled_df['charges']

In [237]:
KNN = KNeighborsClassifier(n_neighbors=3)
cv_scores = cross_val_score(KNN,X,y,cv=10)
print(cv_scores)
print(cv_scores.mean())

[0.94776119 0.92537313 0.95522388 0.91791045 0.93283582 0.93283582
 0.91044776 0.91791045 0.93233083 0.91729323]
0.9289922567613061


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim